In [2]:
import requests
import json
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from pathlib import Path

In [3]:
# requisicao dos dados
URL = 'https://dadosabertos.camara.leg.br/api/v2/deputados'
resposta = requests.request("GET", URL)

objetos = json.loads(resposta.text) # pega somente os dados dos deputados e descarta o resto

In [4]:
deputados = objetos['dados']    # pega os dados do deputado (lista de dicionarios)

df = pd.DataFrame(deputados)    # transforma cada dicionario em um dataFrame

df = df.drop(['uri', 'uriPartido', 'urlFoto', 'email'], axis=1)    # exclui o id do deputado, pois iremos adicionar um id do banco

In [7]:
df

PL               76
PP               58
PT               56
UNIÃO            51
PSD              46
REPUBLICANOS     44
MDB              37
PSB              24
PSDB             22
PDT              19
PODE              9
NOVO              8
SOLIDARIEDADE     8
PCdoB             8
PSC               8
PSOL              8
CIDADANIA         7
AVANTE            6
PATRIOTA          5
PROS              4
PV                4
PTB               3
REDE              2
Name: siglaPartido, dtype: int64

In [ ]:
coluna_partido = df['siglaPartido'] # dividir em duas tabelas

# transformar todas as strings da siglaPartido em um id para referenciar em uma foreing key
le = LabelEncoder()
coluna_id_partido = le.fit_transform(coluna_partido)

# transforma em listas do python para poder 
coluna_id_partido_list = coluna_id_partido.tolist() # from nparray to list
coluna_partido_list = coluna_partido.values.tolist() # from pandas.series to list

In [ ]:
# troca os dados do dataFrame original pelos ids das siglaPartido
df = df.replace(coluna_partido_list, coluna_id_partido_list)

# converter para str para facilitar subir para o banco
for col in df.columns:
    df[col] = df[col].apply(str)

df = df.rename(columns={'siglaPartido': 'partidoId'})

# salva em um arquivo csv para exportar para o banco
filepath = Path('deputados.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)

df

,id,nome,partidoId,siglaUf,idLegislatura
0,204521,Abou Anni,22,SP,56
1,204379,Acácio Favacho,2,AP,56
2,204560,Adolfo Viana,14,BA,56
3,204528,Adriana Ventura,3,SP,56
4,121948,Adriano do Baldy,9,GO,56
...,...,...,...,...,...
508,178889,Zé Carlos,16,MA,56
509,204559,Zé Neto,16,BA,56
510,160632,Zé Silva,21,MG,56
511,204517,Zé Vitor,7,MG,56


In [ ]:
# para criar outro csv com a tabela partidos
d = dict()  # dicionario necessario para salvar sem duplicar os id e nome do partido para o csv

# cria o dicionario
for i in range(len(coluna_id_partido_list)):
    d[coluna_id_partido_list[i]] = coluna_partido[i]

# cria um dataFrame de acordo com o dicionario e organiza ele em ordem crescente pelo id
df_partido = pd.DataFrame(list(d.items()), columns=['id', 'siglaPartido']).sort_values(['id'])

# salva em um arquivo csv para exportar para o banco
filepath = Path('partidos.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_partido.to_csv(filepath, index=False)